In [1]:
import matplotlib
matplotlib.use("nbagg")

import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from utility import *
from model import *

from os import listdir
from os.path import isfile, join
import numpy as np
import math

SyntaxError: invalid syntax (utility.py, line 342)

# Define from where to load the files 

In [ ]:
polycube_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/Generated-Cars-Grid_size100/"
polycube_files = [f for f in listdir(polycube_path) if isfile(join(polycube_path, f))]

voxelized_mesh_path = "/Users/davidcleres/DeepShape/Polycubing-Automated/voxelizedMeshes-Grid_size100/"
voxelized_mesh_files = [f for f in listdir(voxelized_mesh_path) if isfile(join(voxelized_mesh_path, f))]

voxelizedFiles = []
polycubedFiles = []

for f in voxelized_mesh_files: 
    if f[-13:] == "voxelized.txt":
        voxelizedFiles = np.hstack((voxelizedFiles, f))
    
for f in polycube_files:
    if f[-14:] == "finalCubes.txt":
        polycubedFiles = np.hstack((polycubedFiles, f))

# Load the data

In [ ]:
grid_size = 100
voxelized_train_input, polycube_target=loadData(grid_size, polycube_path, voxelized_mesh_path, voxelizedFiles, polycubedFiles, loadFromScratch=False)

# Build a training and validation set 

In [ ]:
batch_size = 5 
preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target = preprocessing_train(voxelized_train_input, polycube_target,batch_size, False, False)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train)
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation)
preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:, 0,0,0,0]) 
Nvalidation = len(preprocessed_input_validation[:,0,0,0,0])
image_size = 100

train_input = np.array(preprocessed_input_train.view(Ntrain, 1, image_size, image_size, image_size))
validation_input = np.array(preprocessed_input_validation.view(Nvalidation, 1, image_size, image_size, image_size))

labels_train = np.array(preprocessed_input_train_target.view(Ntrain, 1, image_size, image_size, image_size))
labels_validation = np.array(preprocessed_input_validation_target.view(Nvalidation, 1, image_size, image_size, image_size))

print('train', train_input.shape)
print('validation', validation_input.shape)
print('train_target', labels_train.shape)
print('validation_target', labels_validation.shape)

# Make test for one cube 

In [ ]:
delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z = find_center_and_delta(labels_train[i, 0, :, :, :], grid_size=100)
outputDC = build_cube(delta_x_left, delta_x_right, center_x, delta_y_left, delta_y_right, center_y, delta_z_left, delta_z_right, center_z, grid_size)

In [ ]:
voxels = np.array(labels_train[3,0,:,:,:])

# and plot everything
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.voxels(voxels)
fig.savefig('VoxelizedFinal.png')
fig.show()

In [ ]:
voxels = np.array(outputDC)

# and plot everything
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.voxels(voxels)
fig.savefig('VoxelizedFinal.png')
fig.show()

In [ ]:
voxels = np.array(outputDC-labels_train[3,0,:,:,:])

# and plot everything
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
ax.voxels(voxels)
fig.savefig('VoxelizedFinal.png')
fig.show()

In [ ]:
nb_errors = 0
Ndata = len(data_input[:, 0, 0, 0, 0])

print("batch_output", batch_output.shape)
print("data_input.narrow(0, b_start, bsize_eff)",  data_input.narrow(0, b_start, bsize_eff).shape)

if len(list(test_prediction.size()))>1 and test_prediction.size(1) > 1: #if CrossEntropy
    # as many ouputs as there are classes => select maximum output
    nb_errors += (test_prediction.max(1)[1] != labels_validation).long().sum()
    # overflow problem if conversion to Long Int not performed, treated as short 1-byte int otherwise!!

else: #if BCE --> if >0.5 or of <0.5
    # output is a scalar in [0, 1]
    err_matrix = test_prediction.round()-data_target #should give 0 if same and ±1 if not the same
    nb_errors += np.sum(np.abs(err_matrix).view(-1))

print("nb_validation_errs", nb_errors)

print("Epoch Number : ", i_ep)
print("\t Validation accuracy ", np.abs(100*(Nvalidation*64*64*64-int(nb_errors))/(Nvalidation*64*64*64))) 
